# Initializations

In [ ]:
import re
import random
import numpy as np
from copy import deepcopy
from datetime import datetime

# Timeout in seconds for an observation run. 
# The longer the time period, the more accurate the probabilities and the diagnoses. 
# The resulting text file shows a status per observation if the program could succeed in completing the process or not.
TIMEOUT_PER_OBSERVATION     = 30*60

USE_GOOGLE_DRIVE_FOR_FILES    = True

if USE_GOOGLE_DRIVE_FOR_FILES:
  from google.colab import drive
  drive.mount('/content/drive')

  DATA_FOLDER_PATH              = "/content/drive/My Drive/Data Science/BGU/Anomaly Detection and Diagnosis/Project/Program Files/"

else:
  DATA_FOLDER_PATH = ""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Gates

In [ ]:

class Gate:
  """
  A logical gate in the system

  Parameters
  ----------
  action:                  str,
                           the gate type. supported types:  'and', 'buffer', 'inverter', 'nand', 'nor', 'or', 'xor'.
  gate_size:               int,
                           the number of inputs to the gate.
  name:                    str,
                           the name of the gate as it appears in the file.
  input_nicknames:         str,
                           the input names as they appear in the file.
  output_nickname:         str,
                           the output name as it appears in the file.                         
  """
  def __init__(self, action, gate_size, name, input_nicknames, output_nickname, p_gate):
    self.__action = action
    self.__gate_size = gate_size
    self.__name = name
    self.__input_nicknames = input_nicknames
    self.__output_nickname = output_nickname
    self.__p_gate_faulty = p_gate
    self.__input_indexes = []
    self.__output_index = None
    self.__rank = None
    self.__processor = {}
    self.__faulty = False
    self.initialize_dictionary()
   


  def initialize_io(self, signals_names):
    """
    initalizes the input and output indexes of the given signals_names.
    e.g: for c17, i1,i3 will be initailized as [0,2]
    """
    for name in self.__input_nicknames:
      self.__input_indexes.append(signals_names.index(name))
    self.__output_index= signals_names.index(self.__output_nickname)
    


  def initialize_dictionary(self):
    """
    creates a dictionary of all possible values for the gate, for fast processing during runtime.
    """
    found = True
    dict_values = initialize_permutations(self.__gate_size)
    
    for dict_key_values in dict_values:
      dict_key_values_str = [str(i) for i in dict_key_values]
      final_key = "".join(dict_key_values_str)
      if self.__action == "and":
        result = 0 if '0' in final_key else 1
      elif self.__action == "buffer":
        result = 0 if final_key=='0' else 1
      elif self.__action == "inverter":
        result = 0 if final_key=='1' else 1
      elif self.__action == "nand":
        result = 1 if '0' in final_key else 0
      elif self.__action == "nor":
        result = 0 if '1' in final_key else 1
      elif self.__action == "or":
        result = 1 if '1' in final_key else 0
      elif self.__action == "xor":
        if self.__gate_size > 2:
          raise ValueError(f"missing initialization for {self.__action}{self.__gate_size}")
        result = 0 if final_key in ['00', '11'] else 1
      else:
        raise ValueError(f"missing initialization for {self.__action}")
      
      final_key = '[' + ", ".join(final_key) + ']'
      self.__processor[final_key] = result



  def process(self, inputs_arr):
    """
    the method which is called during runtime, to calculate the gate's output.
    inputs_arr - an array with the inputs signals.
    """
    ret = self.__processor[str(inputs_arr)]
    return ret ^ (1 if self.__faulty else 0)



  def __str__(self):
    """
    prints the gate properties.
    """
    print(f"action:\t\t {self.__action}")
    print(f"gate_size:\t {self.__gate_size}")
    print(f"name:\t\t {self.__name}")
    print(f"input_nicknames: {self.__input_nicknames}")
    print(f"output_nicknames: {self.__output_nickname}")
    print(f"input_indexes:\t {self.__input_indexes}")
    print(f"output_index:\t {self.__output_index}")
    print(f"rank:\t\t {self.__rank}")
    return ""

  @property
  def input_nicknames(self):
    return self.__input_nicknames

  @property
  def output_nickname(self):
    return self.__output_nickname

  @property
  def input_indexes(self):
    return self.__input_indexes

  @property
  def output_index(self):
    return self.__output_index

  @property
  def rank(self):
    return self.__rank

  @property
  def p_gate_faulty(self):
    return self.__p_gate_faulty

  @rank.setter
  def rank(self, rank):
    self.__rank = rank

  @property
  def name(self):
    return self.__name

  @property
  def faulty(self):
    return self.__faulty

  @faulty.setter
  def faulty(self, faulty):
    self.__faulty = faulty

 

def initialize_gates(system:str, p_gate):
  """
    reads the system file and initalizes the gates and the signals vector.

    p_gate      - the probability of the gate to be faulty.


    Returns:
    gates       - list of Gate object representing the system gates
    signals     - list of the system signals (in/out), with elements initialzied as None. 
    inputs_num  - the number of inputs in the signals list, starting from index 0.
    outputs_num - the number of outputs in the signals list, starting from index inputs_num.
    
  """

  gates=[]
  signals = []
  signals_names = []

  filename_sys = DATA_FOLDER_PATH + system + '.sys'

  with open(filename_sys) as f:
    content = f.read()

  content_list = (content.replace("\n","")).split(".")

  ###############################################################################################
  # load gates
  ###############################################################################################
  items = content_list[3].split('],[')

  for gate in items:
    
    l = (gate.replace("[","").replace("]","")).split(',')

    # extract action and inputs_len (e.g: 'nand2')
    gate_logic_type = l[0]
    
    gate_action = re.findall("[a-z]+",gate_logic_type)[0]
    if gate_action not in ('and', 'buffer', 'inverter', 'nand', 'nor', 'or', 'xor'):
      raise ValueError("Missing gate validation: ",gate_action)

    if len(gate_action) != len(gate_logic_type):
      # there's a number
      gate_inputs_len = int(re.findall("[0-9]+",gate_logic_type)[0])
    else:
      gate_inputs_len = 1

    gate_inputs_nickname = []  
    for i in range(gate_inputs_len):
      gate_inputs_nickname.append(l[3+i])

    # Gate object creation
    gates.append(Gate(action=gate_action, gate_size=gate_inputs_len, name=l[1], input_nicknames=gate_inputs_nickname, output_nickname=l[2], p_gate=p_gate))


  ###############################################################################################
  # initialize the signals array with inputs, outputs and z-values
  ###############################################################################################
  # inputs (e.g: i1,i3)
  items = (content_list[1].replace('[',"").replace(']',"")).split(',')
  inputs_num = len(items)
  for i in items:
    signals_names.append(i)
    signals.append(None)

  # outputs (e.g: o1)
  items = (content_list[2].replace('[',"").replace(']',"")).split(',')
  outputs_num = len(items)
  for i in items:
    signals_names.append(i)
    signals.append(None)

  # z values (intermediate gates, e.g: z4,z2)
  for gate in gates:
    # what's missing are the gate's inputs and outputs which are not already in signals
    missing = ( set(gate.input_nicknames) | {gate.output_nickname} ) - set(signals_names)
    for item in missing:
      signals_names.append(item)
      signals.append(None)

  ###############################################################################################
  # initailize the gates inputs to their indexes in the signal array
  ###############################################################################################
  for gate in gates:
    gate.initialize_io(signals_names)

  ###############################################################################################
  # rank the gates
  ###############################################################################################

  # simulate 0 inputs
  for i in range(inputs_num):
    signals[i] = 0

  rank=0
  while None in signals:

    for gate in gates: 
      
      # retrieve the gate's signals according to its input indexes
      in_signals = [signals[i] for i in gate.input_indexes]

      # all signals arrived
      if None not in in_signals and gate.rank == None:
        gate.rank = rank
        
    # set the output for all the gates in this rank
    for gate in gates:
      if gate.rank == rank:
        signals[gate.output_index] = 0 # simulate 0 output
    
    rank+=1
    
  # reset
  for i in range(len(signals)):
    signals[i] = None

  return gates, signals, inputs_num, outputs_num
  




def initialize_observations(system:str, gates, signals_len, inputs_num, outputs_num):
  """
  
  reads the observations file and returns the io vector.

  """

  filename_dat = DATA_FOLDER_PATH + system + '_iscas85.obs'
  with open(filename_dat) as f:
    content = f.read()

  content_list = (content.replace("\n","")).split(".") # separate the lines
  if content_list[-1] == "":
    content_list = content_list[:-1]
  content_list = [item[item.find('[')+1:item.find(']')] for item in content_list] # isolate the io vector 
  content_list = [item.split(",") for item in content_list] # create a list from the textual vector

  # initialize the io vectors
  io_vectors = [[0 if i[0]=='-' else 1 for i in item] for item in content_list] # present in binary 0/1

  return io_vectors



def initialize_noise(outputs_num, print_result=False):
  """
  
  initializes random noise for the system's outputs.

  """
  ret = [0.01 * random.randint(1,10) for i in range(outputs_num)]
  if print_result:
    for i,p in enumerate(ret):
      print(f"Out{i}={p}, ", end="")
    print("\n")
  return ret

          

def initialize_permutations(outputs_num):
  """
  
  returns a list with all possible outputs combintation (e.g: [0,0], [0,1], [1,0], [1,1]).

  """
  res = []
  temp = []
  for i in range(pow(2,outputs_num)):
    s=("0"*outputs_num + bin(i).replace("0b", ""))[-outputs_num:]
    temp.append(list(s))
  for item in temp:
    res.append([int(i) for i in item])
  return res




def initialize_ranks(gates):
  """
  
  returns a list of lists. 
  The external list i-th index represent the i-th rank in the system.
  The internal list contains the gate index in the gates list.

  """
  gate_rank_indexes = []
  max_rank = max([i.rank for i in gates])
  
  for i_rank, rank in enumerate(range(max_rank+1)):
    gate_rank_indexes.append([])
    for i_gate, gate in enumerate(gates):
      if gate.rank == rank:
        gate_rank_indexes[i_rank].append(i_gate)
      
  return gate_rank_indexes, max_rank




def get_gates_sequence(gates_queue_method, gates, gate_rank_indexes):
  """
  
  returns a list which is used as a queue of nodes for the dfs method.
  the list represents the sequence in which the gates should be processed during the tree creation.
  gates_queue_method - 'SERIAL' - according to the order in the ISCAS-85 file. Left most node is 0.
                       'RANKED' - considers the model gate ranks. The higher the proximity to the outputs, the higher the rank. 
                                  high ranked gates are accessed first. 

  """
  if gates_queue_method == 'RANKED':
      return [subitem for item in gate_rank_indexes[::-1] for subitem in item]
  elif gates_queue_method == 'SERIAL':
    return list(np.arange(len(gates)))
  raise ValueError('Unknown queue method: ', gates_queue_method)


# Tree Node

In [ ]:

class Node:
  """
  A DFS tree node

  Parameters
  ----------
  path:                    list of gate indexes
  branch_out_success:      list in the length of 2^outputs. The i-th index represent the status of the i-th output permutation. 
                           statuses: 0 = diagnoses not found yet for this output
                                     1 = diagnoses for this output found in this iteration
                                     2 = diagnoses for this output found in previous iterations
  n_gates:                 int, 
                           the number of gates in the system
  n_outputs:               int, 
                           the number of outputs in the system

  """
  def __init__(self, path, branch_out_success, n_gates, n_outputs, print_interval=1_000_000):
    global run_counter
    global run_timer
    run_counter+=1
    
    self.__path = path

    self.__n_gates = n_gates 
    self.__n_outputs = n_outputs   
    self.__nodes = []
    self.__out_success = [2 if item==1 else item for item in branch_out_success]

    if run_counter%print_interval==0:
      print(f"Node {run_counter}, {datetime.now()-run_timer}, {path}")
      run_timer = datetime.now()



  def dfs(self, full_path, prune_level, pre_diagnosis, pre_diagnosis_size, gates, gate_rank_indexes, inputs_len, outputs_len, signals, gates_input_indexes, timeout_per_observation, initial_time):
    """
    
    A recursive method. Evaluates for the current path and stores successful statuses (diagnoses found).
    
    full_path               - a complete list of the gates sequence in the tree.
    prune_level             - int. if bigger than -1, the tree will return upon reaching the prune_level depth.
    pre_diagnosis           - a list of diagnoses per output. 
                              when supplied, the method will look in it to find if it includes the recent pre_diagnosis_size nodes.
                              if it finds, it marks it in the out_success list, which increases the chance of early termination.
    pre_diagnosis_size      - the number of recent nodes to examine when pre_diagnosis is provided.
    gates                   - the system gates list.
    gate_rank_indexes       - list with the ranks of gates. see more details in the run_program method.
    inputs_len              - the number of inputs.
    outputs_len             - the number of outputs.
    signals                 - the signals vector.
    gates_input_indexes     - a list of lists. the i-th element holds the input indexes of gate i. 
    timeout_per_observation - a timeout in seconds for the observation run. 
    initial_time            - the time the dfs (root) began to run.

    Returns:
    True if it reaches timeout, or False otherwise.
    """

    global early_terminations

    # in case a pre-diagnoses has been run, look in it for the new nodes in the path
    if pre_diagnosis and random.random()<0.1:
      
      c = set(self.__path)
      
      for output_index, status in enumerate(self.__out_success):
        if status == None:
          for pd_item in pre_diagnosis[output_index]:
            if pd_item <= c: # additional 'if', to eliminate the need for 2 checks
              if pd_item < c:
                self.__out_success[output_index] = 2
              else:
                self.__out_success[output_index] = 1


    if all(self.__out_success):
      early_terminations+=1
      return False
      

    if self.__path: 
      # not root

      # find the output vector, given the faulty gates
      result = evaluate(gates, gate_rank_indexes, inputs_len, outputs_len, signals, faulty_gates=self.__path,  gates_input_indexes=gates_input_indexes)

      # caluclate binary to decimal, to find the permutation index in the out_success array
      result_int = sum([out_result * pow(2, (self.__n_outputs-1) - index) for index, out_result in enumerate(result)])

      if self.__out_success[result_int] != 2:
        # first time for this branch
        self.__out_success[result_int] = 1

      # when all the outputs have been diagnosed for this path, no need to continue the diagnosis
      if all(self.__out_success):
        early_terminations+=1
        return False

      # the last item in the path is the last in the full_path of all the gates
      if self.__path[-1]==full_path[-1]:
        return False
      
      next_index = full_path.index(self.__path[-1])+1
        
    else:
      next_index = 0
 
    # prune
    if len(self.__path)==prune_level:
      return False

    # go deeper. begin with the next number after the last one in path
    for i_element, element in enumerate(full_path[next_index:]):

      self.__nodes.append(Node(self.__path + [element], self.__out_success, self.__n_gates, self.__n_outputs))

      reached_timeout = self.__nodes[-1].dfs(full_path, prune_level, pre_diagnosis, pre_diagnosis_size, gates, gate_rank_indexes, inputs_len, outputs_len, signals, gates_input_indexes, timeout_per_observation, initial_time)
      
      if reached_timeout or (datetime.now() - initial_time).seconds >= timeout_per_observation: 
        return True

    return False




  def get_out_results(self, all_out, result):
    
    """
    Returns the results of the dfs run.

    all_out - list with all the output permutations.
    result  - container. received as empty.

    Returns:
    when the run finishes, the result variable will be a list of all the tree nodes. each item
    will be a tuple of (<path>, <output>). 
    <path> indicates the path to the node (faulty gates).
    <output> indicates the output signal.
    """

    result.append((self.__path, [i[0] for i in zip(all_out, self.__out_success) if i[1]==1]))
    for n in self.__nodes:
      n.get_out_results(all_out, result)
    
  def print_me(self):
    """
    prints the tree
    """
    print(f"path: {self.__path}, out_success: {self.__out_success}")
    for n in self.__nodes:
      n.print_me()

  @property
  def nodes(self):
    return self.__nodes

  @property
  def path(self):
    return self.__path
  
  @property
  def out_success(self):
    return self.__out_success


def tree_find_last_search(nd):
  """
    returns the last path explored in the tree. 
    this function can be used to determine the search coverage in cases when timeout is being activated and the dfs stops in the middle of the run.
  """
  while nd.nodes:
    nd = nd.nodes[-1]
  return nd.path

# Evaluations

In [ ]:

def evaluate(gates, gate_rank_indexes, inputs_len, outputs_len, signals, faulty_gates, gates_input_indexes):

    """
    returns the system's output, given the inputs.

    gates               - list of the system gates.
    gate_rank_indexes   - ist with the ranks of gates. see more details in the run_program method.
    inputs_len          - the number of system inputs.
    outputs_len         - the number of system outputs.
    signals             - the system's signals vector.
    faulty_gates        - a list of the faulty gates indexes.
    gates_input_indexes - a list of lists. the i-th element holds the input indexes of gate i. 

    Returns:
    a list with the system's output signals.
    """

    # reset outputs and intermediate inputs. keep the inputs.
    signals[inputs_len:] = [None]* (len(signals)-inputs_len)
    
    # initialize faulty gates
    for gate in gates:
      gate.faulty = False
    for i in faulty_gates:
      gates[i].faulty = True
      
    
    # evaluation is done rank by rank, to get the intermediate z signal results
    for gate_rank_indexes_i in gate_rank_indexes:
      for item in gate_rank_indexes_i:

        # load the input signals. To find their location in signals, use their indexes which are indicated in the gate.
        in_signals = [signals[i] for i in gates_input_indexes[item]] 
       
        # process the gate, and place its result on the out signals
        signals[gates[item].output_index] = gates[item].process(in_signals) 


    return signals[inputs_len:inputs_len+outputs_len]




def calculate_diagnosis_probabilities(diagnosis, obs_out, p_noise, all_out, p_gate):
  
  """
  Returns the diagnosis probabilities for the given parameters.
  
  diagnosis - a list of list. each element i in the outer list is a list of diagnoses for the ouput permutation i.
              (e.g: in a 4-outputs system diagnosis[2] will hold the diagnosis of the output obseravtion [0,0,1,0])
  obs_out   - the observation output vector as given in the file.
  p_noise   - a vector of probabilities of the noise per output.
  all_out   - list with all the output permutations.
  p_gate    - the probability of a gate to be faultuy.

  Returns a list of probabilities per output permutation.

  """
  
  res = []

  # because of the noise, we have to test each output as a possible output for the system.
  # for each output, we will assume that it is the "correct" system output, and then that it was changed by the noise to become the 
  # observation output. For example: if the observation is [1,1], we will find for each output [[0,0],[0,1],[1,0],[1,1] the probability 
  # that the became [1,1] due to the noise ratios.
  for out_index, out in enumerate(all_out):

    # initialize with True or False by comparing to the observation
    is_noise = [out[i]!=obs_out[i] for i in range(len(out))]
    
    # calculate probability to noise (or no noise) for each output
    p_noise_per_output = [item[0] if item[1]==True else 1-item[0] for item in zip(p_noise,is_noise)]
    
    # mutiply the probabilities, to get the total out probablity (the probability of the "out" state to become obs_out, due to noise)
    p_noise_total = 1
    for i in p_noise_per_output:
      p_noise_total = p_noise_total * i

    # for each element in the diagnosis, calculate it's probablity to be faulty
    p_d = []
    for d in diagnosis[out_index]:
      p_d.append (pow(p_gate, len(d)))

    # normalize
    sub_res = []
    if sum(p_d) > 0:
      factor = 1 / sum(p_d)
      p_d = [i * factor for i in p_d]
      
      # the diagnosis probability equals to the probability of its gates to be faulty and the output to become the observation output
      for item in p_d:
        sub_res.append(item * p_noise_total)

    res.append(sub_res)

  return res


def collect_diagnoses(root, all_out, pre_diagnosis, pre_diagnosis_level):
  
  """
  Scans the tree to collect the diagnoses.

  root                  - Node. the dfs tree root.
  all_out               - list with all the output permutations.
  pre_diagnosis         - a list of diagnoses per output. 
                          when supplied, the method will merage it with the tree results.
  pre_diagnosis_level   - the maximal size of diagnosis in the pre_diagnosis list.
  """

  ret = []
  candidates = []
  full_sets = []

  root.get_out_results(all_out, candidates)

  for o in all_out:
    full_sets.append(sorted([diagnosis for diagnosis, out in candidates if out==[o]], key=lambda x:len(x)))

  if pre_diagnosis:

    # convert pre_diagnosis from set to list
    dg_list =[]
    for item in pre_diagnosis:
      dg_list.append([list(subitem) for subitem in item])
    
    for i in range(len(all_out)):

      # remove the pre diagnosis from the full set (if exists)
      small_size_diagnoses = [item for item in full_sets[i] if len(item)<=pre_diagnosis_level]
      
      for pr in pre_diagnosis:
        if pr in small_size_diagnoses:
            full_sets[i].remove(pr)
      
  # remove all super sets from within the outputs.
  # on each full set, run per diagnosis size (size of set)
  for full_set in full_sets:
    
    if full_set:

      full_set = [set(i) for i in full_set]
      new_full_set = []
      items_lengths = [len(i) for i in full_set]
      max_len = max(items_lengths)
      
      # size 1 has to run on 2 and above, then 2 has to run on 3 and above, etc..
      for size in range(1,max_len):

        current_size_items = [i for i in full_set if len(i)==size]
        next_sizes_items = [index for index, item in enumerate(full_set) if len(item)>size]
        
        for x in current_size_items:
          for y in next_sizes_items:
            if x & full_set[y] == x:
              full_set[y] = set()

    ret.append([i for i in full_set if i])
    
    
  return ret

   

# Print and Log Utilities

In [ ]:

def print_diagnoses(diagnoses, diagnoses_p, all_out):
  """
  A utility method to return a formatted string of the given diagnosis.
  """

  text = "Diagnoses:\n"

  for i in range(len(all_out)):
    text += f"\nOutput {all_out[i]}:\n"

    for i_d, d in enumerate(diagnoses[i]):
      text += f"{d}, P={diagnoses_p[i][i_d]}\n"  

  return text



def print_results(final:bool, header:bool, to_screen:bool, to_file:bool, save_log:bool, system_name, gates, single_observation:bool, observations, 
                  observation_number, diagnoses, diagnoses_p, diagnoses_stats, p_noise, all_out, p_gate, duration, cache_used_counter, last_search_path, file_name_extension, pre_diagnosis_size):
  
  """
  A utility method to return a formatted string of the results.
  """


  text = ""
  file_name = f"{DATA_FOLDER_PATH}_{system_name}_{file_name_extension}_Pre_{pre_diagnosis_size}_{f'diagnoses_{datetime.now().isoformat()}' if final else 'temp'}.txt"
  
  if header:

    file_mode = 'w'

    if final:
        
      sec = duration.total_seconds()
      time_text = "Run completed in "
      if sec >= 60:
        time_text += f"{sec//60} minute" + ("s" if sec>=120 else "") + " and "
      time_text += str(round((sec - (sec//60) * 60),1)) + " seconds"

    else:
      time_text =""
      
      
    # title with system properties
    title = f"DIAGNOSIS RESULTS FOR SYSTEM {system_name.upper()}"
    text = f"\n{'#'*len(title)}\n{title}\n{'#'*len(title)}\n\n"
    if time_text:
      text+=f"{time_text}\n\n"
    text += "Gate indexes:\n"
    for i,gate in enumerate(gates):
      text += f"{i} = {gate.name}\n"

    text += f"\nGate probability to be faulty:\n{p_gate}\n"

    text += "\nNoise probability:\n"
    for i,p in enumerate(p_noise):
      text += f"Out{i} = {p}\n"

    text += f"\nPre Diagnosis Size: {pre_diagnosis_size}\n"

  else:
    file_mode = 'a'

  if single_observation:
        
    
    sub_title = f"Observation #{observation_number+1} - {observations}:"
    text += f"\n\n{sub_title}\n{'-'*len(sub_title)}\n\n"

    sec = duration.total_seconds()
    time_text = "Observation run completed in "
    if sec >= 60:
      time_text += f"{sec//60} minute" + ("s" if sec>=120 else "") + " and "
    time_text += str(round((sec - (sec//60) * 60),1)) + " seconds"

    text += time_text + "\n\n"

    text += print_diagnoses(diagnoses, diagnoses_p, all_out)

  else:

    if observations:
      
      average_early_terminations = np.average([diagnoses_stats[i][3] for i in diagnoses_stats.keys()])

      # number of observations and cache usage
      text += f"\nNumber of observations: {len(observations)}\n"
      text += f"\nAverage Early Terminations {average_early_terminations:.2f}\n"
      text += f"\nResults from cache have been used {cache_used_counter} times\n"
      text += "\n"

      # general stats per observation and timeouts
      text += f"Timeouts\n--------\n\n"
      text += f"Timeout per Observation: {TIMEOUT_PER_OBSERVATION} seconds\n"
      reached_timeout_count = len([i for i in diagnoses_stats.keys() if diagnoses_stats[i][0]])
      average_total_diagnoses = np.average([diagnoses_stats[i][1] for i in diagnoses_stats.keys()])
      average_diagnoses_per_output = average_total_diagnoses / len(all_out) #np.average([diagnoses_stats[i][2] for i in diagnoses_stats.keys()])
      average_diagnoses_size = np.average([len(subitem) for sublist in diagnoses for item in sublist for subitem in item])
      
      text += f"Reached Timeout: {reached_timeout_count}/{len(diagnoses_stats.keys())} \
                \nAverage Diagnoses per Observation: {average_total_diagnoses:.2f} \
                \nAverage Diagnosis Size: {average_diagnoses_size:.2f}        \
                \nAverage Diagnoses per Output: {average_diagnoses_per_output:.2f}\n\n"

      text += f"Observation #  -  Reached Timeout  -  Total Diagnoses  -  Average Diagnoses per Output  - Average Diagnosis Size\n"
      for i in diagnoses_stats.keys():
        text += f"{(i+1):4.0f}\t-\t{diagnoses_stats[i][0]}\t-\t{diagnoses_stats[i][1]}\t-\t{diagnoses_stats[i][2]:.1f}\t-\t{np.average([len(item) for sublist in diagnoses[i] for item in sublist]):.2f}\n"
    
      for i, observation in enumerate(observations):
        sub_title = f"Observation #{i+1} - {observation}:"
        text += f"\n\n{sub_title}\n{'-'*len(sub_title)}\n\n"
        text += print_diagnoses(diagnoses[i], diagnoses_p[i], all_out)


      text += f"\nLast search path:\n{last_search_path}"

  if to_screen:  
    print(text)
  
  if to_file:

    with open(file_name, file_mode) as f:
      f.write(text)
  
  if save_log:
    file_name = system_name + "_diagnoses_log_" + datetime.now().strftime("%Y-%m-%d %H-%M-%S") + ".txt"
    with open(file_name, 'w') as f:
      for item in db:
        f.write(str(item))
        f.write("\n")
  



# Main

In [ ]:
run_timer   = datetime.now()
run_counter = 0
system      = '74182_short' # '74283_short', 'c17', '74182_short', 'c432_short', 'c3540_short'


run_program(system, 'RANKED', pre_diagnosis_level=1)


In [ ]:

def run_program(system, gates_queue_method, pre_diagnosis_level):
  """
  Runs the program

  Parameters
  ----------
  system:                  str,
                           name of the iscas-85 system 
  gates_queue_method:      str,
                           the default method in which the gates will be ordered in the DFS tree. 
                           'SERIAL' - according to the order in the ISCAS-85 file. Left most node is 0.
                           'RANKED' - considers the model gate ranks. The higher the proximity to the outputs, the higher the rank. High ranked gates are accessed first. 
  pre_diagnosis_level:     int, 
                           the depth of the pre-diagnosis which is used to construct a the pre-diagnosis tree. see documentation for more details.
  """

  global early_terminations
  start_time = datetime.now()
    
  gates=[]
  signals = []
  all_out = []
  diagnoses = []
  diagnoses_p = []
  gate_rank_indexes = []
  p_gate = 0.01 # the probability of a gate to be faulty

  print("initializing gates...")
  gates, signals, inputs_num, outputs_num = initialize_gates(system, p_gate)
  
  # gates_input_indexes is a list of lists. the i-th element holds the input indexes of gate i
  gates_input_indexes = [g.input_indexes for g in gates]

  # gate_rank_indexes is a list of lists. the i-th element holds the gate numbers of rank i
  print("initializing ranks...")
  gate_rank_indexes, max_rank = initialize_ranks(gates)
  
  print("initializing observations...")
  observations = initialize_observations(system, gates, len(signals), inputs_num, outputs_num)
  
  print("initializing noise probabilities...")
  p_noise = initialize_noise(outputs_num, print_result=True)

  print("initializing all output possibilities...")
  all_out = initialize_permutations(outputs_num)

  print_results(final=False, header=True, to_screen=False, to_file=True, save_log=False, system_name=system, gates=gates, single_observation=False, observations=None, 
                observation_number=None, diagnoses=None, diagnoses_p=None, diagnoses_stats=None, p_noise=p_noise, all_out=all_out, p_gate=p_gate, duration=None, 
                cache_used_counter=None, last_search_path=None, file_name_extension=gates_queue_method, pre_diagnosis_size=pre_diagnosis_level)

  # find the diagnoses and their probabilities
  print(f"Diagnosing {len(observations)} observations using DFS (timeout per observation = {TIMEOUT_PER_OBSERVATION} [sec])...\n")

  diagnoses_cache = {}
  diagnoses_stats = {}
  cache_used_counter = 0

  for observation_i, observation in enumerate(observations):

    start_time_temp = datetime.now()

    print(f"\nObservation {observation_i},", end="")
    print(observation[:inputs_num])
    inputs_str = "".join([str(i) for i in observation[:inputs_num]])
    current_diagnosis = None

    # check existence in cache
    if inputs_str in diagnoses_cache.keys():
      
      current_diagnosis = diagnoses_cache[inputs_str]
      print(f"{observation_i} taken from cache!")
      cache_used_counter += 1

    else:
      
      # insert inputs
      signals = [None] * len(signals)
      obs_in = observation[:inputs_num]
      for i in range(inputs_num):
        signals[i] = obs_in[i]

      
      full_path = get_gates_sequence(gates_queue_method, gates, gate_rank_indexes)
  
      pre_diagnosis_size = pre_diagnosis_level
      pre_diagnosis = []

      if pre_diagnosis_level > 0:
        print(f"Running pre-diagnosis {pre_diagnosis_level}-size)...")

        root = Node(path=[], branch_out_success=[None] * pow(2, outputs_num), n_gates=len(gates), n_outputs=outputs_num)
        root.dfs(full_path, pre_diagnosis_level, [], 0, gates, gate_rank_indexes, inputs_num, outputs_num, signals, gates_input_indexes, TIMEOUT_PER_OBSERVATION, datetime.now())
        
        pre_diagnosis = collect_diagnoses(root, all_out, [], 0)
        print(pre_diagnosis)

      # initialize tree
      root = Node(path=[], branch_out_success=[None] * pow(2, outputs_num), n_gates=len(gates), n_outputs=outputs_num)
      early_terminations = 0
      reached_timeout = root.dfs(full_path, -1, pre_diagnosis, pre_diagnosis_size, gates, gate_rank_indexes, inputs_num, outputs_num, signals, gates_input_indexes, TIMEOUT_PER_OBSERVATION, datetime.now())
      print("early_terminations:", early_terminations)

      current_diagnosis = collect_diagnoses(root, all_out, pre_diagnosis, pre_diagnosis_level)
      
      current_diagnosis_count = [len(l) for l in current_diagnosis]
      diagnoses_stats[observation_i] = [reached_timeout, sum(current_diagnosis_count),np.average(current_diagnosis_count), early_terminations]

      # store in cache
      diagnoses_cache[inputs_str] = current_diagnosis
      
    p = calculate_diagnosis_probabilities(current_diagnosis, observation[inputs_num:], p_noise, all_out, p_gate)
    diagnoses.append(current_diagnosis)
    diagnoses_p.append(p)

    print_results(final=False, header=False, to_screen=False, to_file=True, save_log=False, system_name=system, gates=gates, single_observation=True, observations=observation, observation_number=observation_i, 
                  diagnoses=current_diagnosis, diagnoses_p=p, diagnoses_stats=None, p_noise=p_noise, all_out=all_out, p_gate=p_gate, duration=datetime.now()-start_time_temp, 
                  cache_used_counter=None, last_search_path=None, file_name_extension=gates_queue_method, pre_diagnosis_size=pre_diagnosis_size)

       
  print("\n")

  print("Preparing results...")
  print_results(final=True, header=True, to_screen=False, to_file=True, save_log=False, system_name=system, gates=gates, single_observation=False, observations=observations,
                observation_number=None, diagnoses=diagnoses, diagnoses_p=diagnoses_p, diagnoses_stats=diagnoses_stats, p_noise=p_noise, all_out=all_out, p_gate=p_gate, duration=datetime.now()-start_time, 
                cache_used_counter=cache_used_counter, last_search_path = tree_find_last_search(root), file_name_extension=gates_queue_method, pre_diagnosis_size=pre_diagnosis_size)

  print("Done.")
